In [ ]:
import pandas as pd
import numpy as np
import joblib
import shap
import matplotlib.pyplot as plt
import lightgbm as lgb
from config import MODELS_DIR


In [ ]:
# Load validation data
X_val = joblib.load(MODELS_DIR / "X_val.pkl")
y_val = joblib.load(MODELS_DIR / "y_val.pkl")



In [ ]:
# Load base models
logreg = joblib.load(MODELS_DIR / "logreg_baseline.pkl")
xgb = joblib.load(MODELS_DIR / "xgb_fraud_model_tuned.pkl")
lgb_model = lgb.Booster(model_file=str(MODELS_DIR / "lgb_fraud_model_tuned.txt"))



In [ ]:
# Rebuild meta-features for validation
meta_val = pd.DataFrame({
    "logreg_prob": logreg.predict_proba(X_val)[:, 1],
    "xgb_prob": xgb.predict_proba(X_val)[:, 1],
    "lgb_prob": lgb_model.predict(X_val)
})



In [ ]:
# Load stacking meta-model
meta_model = joblib.load(MODELS_DIR / "stacking_meta_model.pkl")



In [ ]:
# Predict probabilities
val_probs = meta_model.predict_proba(meta_val)[:, 1]